In [ ]:
#-------------------------------------------------------------------TASK 1 -------------------------------------------------------------------

In [1]:
import os
import glob
import shutil
import pandas as pd

In [ ]:
# 3. Change directory to 'csv_files' so glob picks them up
os.chdir("csv_files")

c:\Users\mianf\Desktop\Assigmnetn 5 IT\csv_files


In [8]:
# Move all CSV files to backup folder
csv_files = glob.glob("*.csv")
for file in csv_files:
    shutil.move(file, "../backup_folder")
    print(f"Moved file: {file}")

Moved file: continent.csv
Moved file: language.csv
Moved file: locations.csv


In [9]:
# Automating Export
def export_data(df, filename, format):
    if format == "csv":
        df.to_csv(filename, index=False)
        print(f"Data exported to {filename} in CSV format.")
    elif format == "json":
        df.to_json(filename, orient="records")
        print(f"Data exported to {filename} in JSON format.")
    else:
        print("Unsupported format.")

In [10]:
# Example usage:
# Creating a sample dataframe
data = {
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 30, 35],
    'City': ['New York', 'Los Angeles', 'Chicago']
}
df = pd.DataFrame(data)

# Exporting to CSV
export_data(df, "output.csv", "csv")

# Exporting to JSON
export_data(df, "output.json", "json")

Data exported to output.csv in CSV format.
Data exported to output.json in JSON format.


In [ ]:
#-------------------------------------------------------------------TASK 2 -------------------------------------------------------------------

In [ ]:
!pip install yfinance

In [2]:
import yfinance as yf
import sqlite3
import time

In [3]:
# Database setup
db_name = "stocks.db"
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS stock_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    symbol TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume INTEGER
)
''')
conn.commit()

In [4]:

# Function to fetch stock data
def fetch_stock_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        data = stock.history(period="1d", interval="1m")

        if data.empty:
            print(f"No data found for {symbol}. Skipping...")
            return None

        latest = data.iloc[-1]  # Most recent row
        return {
            "symbol": symbol,
            "open": latest["Open"],
            "high": latest["High"],
            "low": latest["Low"],
            "close": latest["Close"],
            "volume": latest["Volume"]
        }
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Function to store data in SQLite
def store_data(symbol):
    stock_data = fetch_stock_data(symbol)
    if stock_data:
        cursor.execute('''
            INSERT INTO stock_data (symbol, open, high, low, close, volume)
            VALUES (?, ?, ?, ?, ?, ?)
        ''',
        (stock_data["symbol"], stock_data["open"], stock_data["high"],
         stock_data["low"], stock_data["close"], stock_data["volume"]))
        conn.commit()
        print(f"Stored data for {symbol}")

# Function to analyze stock data
def analyze_stock(symbol):
    df = pd.read_sql_query(
        "SELECT * FROM stock_data WHERE symbol=? ORDER BY timestamp DESC LIMIT 100",
        conn,
        params=(symbol,)
    )
    print(df)




In [5]:
# Example Usage
symbol = "INTC" #Stocks for Intel Corporation

for _ in range(5):  # fetch data 5 times at 1-minute intervals
    store_data(symbol)
    time.sleep(60)

analyze_stock(symbol)

# Close database connection
conn.close()

Stored data for INTC
Stored data for INTC
Stored data for INTC
Stored data for INTC


$INTC: possibly delisted; no price data found  (period=1d)


No data found for INTC. Skipping...
   id symbol            timestamp       open       high        low      close  \
0   4   INTC  2025-03-06 17:43:03  20.670000  20.695000  20.660000  20.674400   
1   3   INTC  2025-03-06 17:42:02  20.709900  20.719900  20.670000  20.670000   
2   2   INTC  2025-03-06 17:41:01  20.695000  20.695000  20.695000  20.695000   
3   1   INTC  2025-03-06 17:39:59  20.655001  20.655001  20.655001  20.655001   

   volume  
0   52687  
1   88592  
2       0  
3       0  


In [6]:
#-------------------------------------------------------------------TASK 3 -------------------------------------------------------------------

In [7]:
import requests
from bs4 import BeautifulSoup

In [11]:
URL = "http://books.toscrape.com/"
HEADERS = {"User-Agent": "Mozilla/5.0"}
def get_books(url):
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")
    books = soup.find_all("article", class_="product_pod")
    book_list = []
    for book in books:
        title = book.h3.a["title"]
        price = book.find("p", class_="price_color").text
        stock = book.find("p", class_="instock availability").text.strip()
        book_list.append({"Title": title, "Price": price, "Availability":
        stock})
    return book_list

books_data = get_books(URL)
df = pd.DataFrame(books_data)
df.to_csv("books.csv", index=False)
print("Data saved to books.csv")
print(df)

Data saved to books.csv
                                                Title    Price Availability
0                                A Light in the Attic  Â£51.77     In stock
1                                  Tipping the Velvet  Â£53.74     In stock
2                                          Soumission  Â£50.10     In stock
3                                       Sharp Objects  Â£47.82     In stock
4               Sapiens: A Brief History of Humankind  Â£54.23     In stock
5                                     The Requiem Red  Â£22.65     In stock
6   The Dirty Little Secrets of Getting Your Dream...  Â£33.34     In stock
7   The Coming Woman: A Novel Based on the Life of...  Â£17.93     In stock
8   The Boys in the Boat: Nine Americans and Their...  Â£22.60     In stock
9                                     The Black Maria  Â£52.15     In stock
10     Starving Hearts (Triangular Trade Trilogy, #1)  Â£13.99     In stock
11                              Shakespeare's Sonnets  Â£20.66  